In [ ]:
import os
os.chdir("/home/guest_dyw/diffusion-sampler/")

import neptune
import pandas as pd

import torch
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

import seaborn as sns

sns.set_context("talk")

from utility.loader import load_all_from_experiment_path
from task import get_energy_by_name

energy = get_energy_by_name("ManyWell", "cpu")
project = neptune.init_project(project="dywoo1247/Diffusion-sampler")
run_table = project.fetch_runs_table().to_pandas()


In [95]:
runs = run_table[(run_table["energy"] == "ManyWell") & (run_table["sys/group_tags"] == "ADB") & (run_table["parameters/model/base_std"] != 8.0)]

In [ ]:
runs[["sys/id", "parameters/model/base_std", "parameters/model/trajectory_length"]]

In [97]:
dict = {
    "Exact": energy.sample(2000).to('cpu').norm(dim=-1, p=2)
}

In [ ]:
import math

with torch.no_grad():
    for i, row in runs.iterrows():
        _, model, _, plotter = load_all_from_experiment_path(row["output_dir"])

        param = f"{math.sqrt(row['parameters/model/t_scale']):.1f} {row['parameters/model/trajectory_length']}"

        dict[param] = model.sample(2000).to('cpu').norm(dim=-1, p=2)
        
        print(row['parameters/model/base_std'], row['parameters/model/trajectory_length'])
        sample = model.sample(2000)
        plotter.make_energy_histogram(sample)
        plotter.cfg.sample_figure.fig_size = (12, 7 * 6)
        plotter.cfg.sample_figure.projection_dims=[(0, 2), (2, 4), (4, 6), (6, 8), (8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24)]
        plotter.make_sample_plot(sample)


In [80]:
new_dict = {
    key.replace(' ', '\n'): dict[key]
    for key in dict
}

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.violinplot(data=pd.DataFrame(new_dict))
plt.ylim(6, 12)

plt.grid(True, linestyle='--', alpha=0.3)

plt.ylabel("sample norm")
plt.xlabel("base std\ntrajectory length")

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

import seaborn as sns

sns.set_context("talk")

ax = df[df["std"] == 1.0].plot.line(x="T", y="ELBO", ylim=[120, 164], label="std=1.0", marker="o")

ax = df[df["std"] == 2.0].plot.line(x="T", y="ELBO", ylim=[120, 164], ax=ax, label="std=2.0", marker="o")

ax = df[df["std"] == 4.0].plot.line(x="T", y="ELBO", ylim=[120, 164], ax=ax, label="std=4.0", marker="o")

ax.set_yticks([130, 140, 150, 160])

ax.set_xscale("log")

# Use ScalarFormatter for x-axis labels
ax.xaxis.set_major_formatter(ScalarFormatter())

# Optionally, turn off minor ticks
ax.xaxis.set_minor_locator(plt.NullLocator())

ax.set_xticks([128, 256, 512, 1024])
ax.set_xticklabels([128, 256, 512, 1024])

ax.set_xlabel("trajectory length")
ax.set_ylabel("ELBO")

plt.legend()

# for i, v in enumerate(df[df["std"] == 2.0]["EUBO"]):
#     ax.text(i, v, f"{v:.2f}", ha='center', va='bottom', fontsize=12)

plt.grid(True)

plt.show()